<img src="static/img/PortadaVigas.png" >

# **1. DIAGRAMAS $\sigma$ vs $\varepsilon$**
<img src="static/img/DiagramasMateriales.png">

# **2. TIPOS DE FALLA**
<img src="static/img/tiposDeFalla.png">

# **3. ANÁLISIS DE SECCIONES RECTANGULARES CON FALLA EN TRACCIÓN**

En esta falla se está asumiendo que el acero instalado es menor al acero balanceado y que la deformación unitaria del acero es mayor a la deformación unitaria de fluencia del acero. Esto último se deberá corroborar con los cálculos.

<img src="static/img/fallaTraccion.png">

## Ecuaciones:

- $a=\beta_1 \cdot c$

- $\begin{cases}
                                \beta _{1} =0.85 & ;f'c\leqslant 280\ kg/cm^{2}\\
                                \beta _{1} =1.05-\frac{0.714}{1000} f'c\  & ;280\ kg/cm^{2} \leqslant f'c\leqslant \ 560\ kg/cm^{2}\\
                                \beta _{1} =0.65 & ;f'c\geqslant 560\ kg/cm^{2}
                                \end{cases}$

- $C_c=0.85 \cdot f'c \cdot a \cdot b$

- $T = As \cdot fs = As \cdot fy$  

- $a= \frac{As \cdot fy}{0.85 \cdot f'c \cdot b}$ 

- $Mn = T \cdot (Jd) = As \cdot fy \cdot (d-\frac{a}{2})$    

- Verificar: $\varepsilon_s>\varepsilon_y$                     

# **4. ANÁLISIS DE SECCIONES RECTANGULARES CON FALLA EN COMPRESIÓN**

En este tipo de falla la dificultad radica en que no se conoce el esfuerzo del acero en tracción debido a que este no llega a fluir al momento de la falla. La norma peruana E.060 no permite el uso de secciones sobre reforzadas; sin embargo, las ecuaciones para la resistencia de estas secciones se presentan a continuación.

<img src="static/img/fallaCompresion.png">

- $C_c=T \Longrightarrow 0.85 \cdot f'c \cdot b \cdot a = As \cdot fs \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (a)$

- $\frac{\varepsilon_s}{\varepsilon_{c\mu}}=\frac{d-c}{c} \Longrightarrow \varepsilon_s=0.003 \cdot \frac{d-c}{c}$

- $fs=\varepsilon_s \cdot E_s = 0.003 \cdot \frac{d-c}{c} = 0.003 \cdot E_s \frac{\beta_1 \cdot d - a}{a} \ \ \ \ \ \ \ \ \ \ \ \ \ (b)$

- $\rho = \frac{As}{b \cdot d} \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ (c)$

Sustituyendo $(b)$ y $(c)$ en $(a)$ se obtiene la siguiente ecuación, de la cual se despeja la profundidad del bloque equivalente de compresiones:

$(\frac{0.85 \cdot f'c}{0.003 \cdot E_s \cdot \rho}) \cdot a^2 +a \cdot d - \beta_1 \cdot d^2 = 0$

Finalmente, se tiene la resistencia de la sección:

$Mn = C_c \cdot (Jd) = 0.85 \cdot f'c \cdot a \cdot b \cdot (d-\frac{a}{2})$

Verificar: $\varepsilon_s<\varepsilon_y$

# **5. ACERO MÍNIMO Y MÁXIMO**

$As_{min} = \frac{0.7 \cdot \sqrt{f'c}}{fy} \cdot b \cdot d$

$As_{bal} = b \cdot d \cdot (0.85 \cdot \beta_1 \cdot fc / fy) \cdot \frac{\varepsilon_{c\mu}}{\varepsilon_{c\mu}+\frac{fy}{Es}}$

$As_{max} = 0.75 \cdot As_{bal}$

In [1]:
import math

def calculoFlexion(
    b,
    h,
    fc,
    fy,
    Es,
    Ecu,
    phiFlexion,
    acero,
    r
): 
    
    ####### Cálculos generales #############

    # Cálculo del factor beta1
    if fc <= 280:
        beta1 = 0.85
    elif fc <= 560:
        beta1 = round(1.05 - 0.714 * (fc / 1000), 3)
    else:
        beta1 = 0.65

    # Cálculo de la deformación inicial del concreto comprimido
    E0 = round(Ecu * (1 - beta1), 5)

    ### 1.1 Cálculo de acero minimo############

    d = h - r

    aceroMinimo = 0.7 * ((math.sqrt(fc)) / (fy)) * b * d

    ###1.2 Acero balanceado y acero máximo
    aceroBalanceado = (
        b * d * (0.85 * beta1 * fc / fy) * ((Ecu) / (Ecu + fy / Es))
    )
    aceroMaximo = 0.75 * aceroBalanceado

    if  acero < aceroBalanceado:
        ###1.3 Calculo de a, c y Mn

        ### Asumiendo que el acero fluye (se debe verificar)
        T = acero * fy
        a = T / (0.85 * fc * b)
        c = a / beta1
        Mn = T * (d - a / 2) / (1000 * 100)
        phiMn = phiFlexion * Mn

    else:
        A = (0.85 * fc) / (Ecu * Es * (acero / (b * d)))
        B = d
        C = -beta1 * d * d

        a = (
            -B
            + math.sqrt(B * B - 4 * A * C)
        ) / (2 * A)

        c = a / beta1

        Mn = (0.85 * fc * a * b * (d - a / 2) / (1000 * 100)
        )
        phiMn = phiFlexion * Mn   

    if round(acero,2) < round(aceroBalanceado,2):
        tipoFalla = "Tracción"
    elif round(acero,2) > round(aceroBalanceado,2):
        tipoFalla = "Compresión"
    else:
        tipoFalla = "Balanceada"
        
    defAs = Ecu * (d-c)/c
    
    
    resultado = {}
    resultado["aceroMinimo"] = f'{round(aceroMinimo, 2)} cm^2'
    resultado["aceroBalanceado"] = f'{round(aceroBalanceado, 2)} cm^2'
    resultado["aceroMaximo"] = f'{round(aceroMaximo, 2)} cm^2'
    resultado["a"] = f'{round(a, 2)} cm'
    resultado["c"] = f'{round(c, 2)} cm'
    resultado["Mn"] = f'{round(Mn, 2)} ton-m'
    resultado["phiMn"] = f'{round(phiMn, 2)} ton-m'
    resultado["tipoFalla"] = tipoFalla
    resultado["defAs"] = f'{round(defAs, 5)}'
    resultado["c_value"] = c

    return resultado

In [2]:
import pandas as pd
tablaAceros = pd.DataFrame(
    {
        "Diametro": [
            "6mm",
            '1/4"',
            "8mm",
            '3/8"',
            "12mm",
            '1/2"',
            '5/8"',
            '3/4"',
            '1"',
            '1 3/8"',
        ],
        "Área(cm2)": [0.28, 0.32, 0.5, 0.71, 1.13, 1.29, 2, 2.84, 5.1, 10.06],
    }
)

tablaAceros


,Diametro,Área(cm2)
0,6mm,0.28
1,"1/4""",0.32
2,8mm,0.50
3,"3/8""",0.71
4,12mm,1.13
5,"1/2""",1.29
6,"5/8""",2.00
7,"3/4""",2.84
8,"1""",5.10
9,"1 3/8""",10.06
